In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys, os, pathlib
os.environ['PKG_CONFIG_PATH'] = '/ocean/projects/asc170022p/mtragoza/mambaforge/envs/lung-project/lib/pkgconfig'

import numpy as np
import xarray as xr
import pandas as pd
import nibabel as nib
import torch

sys.path.append('../..')
import project

torch.cuda.is_available()

True

In [3]:
%autoreload
data_root = pathlib.Path('/ocean/projects/asc170022p/mtragoza/lung-project/data/COPDGene')
data_file = data_root / 'sample1000_2025-07-22.csv'

# load sampled data file
df_sampled = pd.read_csv(data_file, sep='\t', low_memory=False)
df_sampled

,sid,ccenter,kernel,Emphysema,pctEmph,pctEmph_Slicer,FEV1pp_utah,FVCpp_utah,FEV1_FVC_utah,finalGold,catEmph
0,16514P,TEM,STD,0,0.3373,0.3147,77.6,88.9,0.71,-1.0,normal
1,20748Q,UAB,STD,0,3.6045,3.4085,73.2,75.8,0.75,-1.0,normal
2,11007Z,USD,STD,0,0.3126,0.2867,66.7,108.0,0.47,2.0,normal
3,14771Z,HPR,STD,0,0.3160,0.2948,94.1,97.0,0.73,0.0,normal
4,13651K,UIA,STD,0,2.0807,1.9551,85.1,88.0,0.75,0.0,normal
...,...,...,...,...,...,...,...,...,...,...,...
995,20519B,DUK,STD,1,30.1942,29.6440,19.9,69.2,0.22,4.0,severe
996,12294H,UAB,STD,1,40.3908,39.5845,43.3,86.0,0.37,3.0,severe
997,23123R,TEM,STD,0,23.6838,22.7617,42.8,72.7,0.45,3.0,severe
998,16546C,UIA,STD,1,42.6116,41.8307,58.6,99.6,0.43,2.0,severe


In [4]:
dataset = project.copdgene.COPDGeneDataset(df_sampled, data_root)
dataset

In [5]:
row, visit = dataset[0]
row

sid               16514P
ccenter              TEM
kernel               STD
Emphysema              0
pctEmph           0.3373
pctEmph_Slicer    0.3147
FEV1pp_utah         77.6
FVCpp_utah          88.9
FEV1_FVC_utah       0.71
finalGold           -1.0
catEmph           normal
Name: 0, dtype: object

In [6]:
variant = 'RAW'
source_name = visit.build_image_name(state='INSP', recon='STD') # moving image
target_name = visit.build_image_name(state='EXP', recon='STD')  # fixed image

$$
    I_{source}(x + u(x)) \approx I_{target}(x)
$$

In [7]:
source_file = visit.image_file(variant, source_name)
target_file = visit.image_file(variant, target_name)
mask_file = visit.mask_file(variant, target_name, mask_name='lung_regions')
mesh_file = visit.mesh_file(variant, target_name, mask_name='lung_regions', mesh_tag='volume')
disp_file = visit.disp_file(variant, source_name, target_name)

In [9]:
def load_nifti_file(nii_file):
    print(f'Loading {nii_file}')
    nifti = nib.load(nii_file)
    print(nifti.header.get_data_shape())
    print(nifti.header.get_zooms())
    print(nifti.affine)
    return nifti.get_fdata(), nifti.header.get_zooms()

source_array, source_affine = load_nifti_file(source_file)
target_array, target_affine = load_nifti_file(target_file)
mask_array, mask_affine = load_nifti_file(mask_file)
disp_array, disp_affine = load_nifti_file(disp_file)

Loading /ocean/projects/asc170022p/mtragoza/lung-project/data/COPDGene/Images/16514P/Phase-1/RAW/16514P_INSP_STD_TEM_COPD.nii.gz
(512, 512, 383)
(0.71875, 0.71875, 0.70000076)
[[  -0.71875      -0.            0.          167.640625  ]
 [   0.            0.71875       0.           -6.640625  ]
 [   0.            0.            0.70000076 -318.50030518]
 [   0.            0.            0.            1.        ]]
Loading /ocean/projects/asc170022p/mtragoza/lung-project/data/COPDGene/Images/16514P/Phase-1/RAW/16514P_EXP_STD_TEM_COPD.nii.gz
(512, 512, 537)
(0.71875, 0.71875, 0.5)
[[  -0.71875    -0.          0.        167.640625]
 [   0.          0.71875     0.         -6.640625]
 [   0.          0.          0.5      -319.      ]
 [   0.          0.          0.          1.      ]]
Loading /ocean/projects/asc170022p/mtragoza/lung-project/data/COPDGene/Images/16514P/Phase-1/RAW/TotalSegmentator/16514P_EXP_STD_TEM_COPD/lung_regions.nii.gz
(512, 512, 537)
(0.71875, 0.71875, 0.5)
[[  -0.71875    

In [21]:
mesh, cell_labels = project.meshing.load_fenics_mesh(mesh_file)
mesh

In [27]:
import fenics as fe

S = fe.FunctionSpace(mesh, 'CG', 1)
V = fe.VectorFunctionSpace(mesh, 'CG', 1)

points = torch.as_tensor(
    S.tabulate_dof_coordinates(), dtype=torch.float, device='cuda'
)
points.shape

torch.Size([13752, 3])

In [31]:
points

tensor([[108.8096, 248.1764,  89.2192],
        [108.8838, 246.2919,  89.0140],
        [108.8561, 247.5657,  91.0147],
        ...,
        [295.7213, 229.8981,  79.1954],
        [287.8640, 239.0472,  82.1989],
        [291.9851, 236.7150,  79.4090]], device='cuda:0')

In [30]:
def world_to_ijk(points, affine):
    A = torch.as_tensor(affine, dtype=points.dtype, device=points.device)
    Ainv = torch.linalg.inv(A)
    ones = torch.ones(points.shape[0], 1, device=points.device)
    pts_h = torch.cat([points, ones], dim=1)
    ijk_h = (A @ pts_h.T).T
    return ijk_h[:,:3]

world_to_ijk(points, mask_affine)

tensor([[  89.4337,  171.7362, -274.3904],
        [  89.3804,  170.3817, -274.4930],
        [  89.4003,  171.2972, -273.4926],
        ...,
        [ -44.9090,  158.5987, -279.4023],
        [ -39.2617,  165.1745, -277.9005],
        [ -42.2236,  163.4983, -279.2955]], device='cuda:0')

In [ ]:
def sample_image_kernel(image, affine, points):

sample_image_kernel(image, affine, points)

In [23]:
import fenics as fe

In [41]:
def solve_fem(
    mesh,
    cell_labels,
    E,
    rho,
    u_obs,
    nu=0.4,
    g=9.81
):
    # function spaces
    S = fe.FunctionSpace(mesh, 'CG', 1)
    V = fe.VectorFunctionSpace(mesh, 'CG', 1)
    
    # Lamé parameters (Pa)
    mu    = E/(2*(1 + nu))
    lmbda = E*nu/((1 + nu)*(1 - 2*nu))

    # kinematics and constitutive law
    def epsilon(u):
        return (fe.grad(u) + fe.grad(u).T) / 2
    
    def sigma(u):
        I = fe.Identity(u.geometric_dimension())
        return lamb*fe.div(u)*I + 2*mu*epsilon(u)

    # body force and traction
    b = fe.as_vector([0, rho*g, 0])

    # variational problem
    u = fe.TrialFunction(V)
    v = fe.TestFunction(V)
    a = fe.inner(sigma(u), epsilon(v))*fe.dx
    L = fe.inner(b, v)*fe.dx

    # boundary conditions
    u_bc = fe.DirichletBC(V, u_obs, 'on_boundary')

    # solve
    u_sim = fe.Function(V)
    fe.solve(a == L, u_sim, [u_bc])

    return u_pred

solve_fem(mesh, 0, 0, 0)

This integral is missing an integration domain.


UFLException: This integral is missing an integration domain.